# Statistics (under work)
v0.5.0
By Stephen Karl Larroque
License: All rights reserved (in the future will be converted to MIT)

In [ ]:
# Forcefully autoreload all python modules
%load_ext autoreload
%autoreload 2

In [ ]:
# AUX FUNCTIONS

import os, sys

cur_path = os.path.realpath('.')
sys.path.append(os.path.join(cur_path, 'csg_fileutil_libs'))  # for unidecode and cleanup_name, because it does not support relative paths (yet?)

import re

from csg_fileutil_libs.aux_funcs import save_df_as_csv, _tqdm, compute_best_diag, reorder_cols_df, find_columns_matching, cleanup_name, replace_buggy_accents, convert_to_datetype, df_drop_duplicated_index, df_to_unicode, df_to_unicode_fast, cleanup_name_df, df_literal_eval, compute_best_diag, df_unify, df_translate, df_filter_nan_str, concat_vals_unique, reorder_cols_df, sort_and_deduplicate, df_squash_lists, df_literal_eval, df_fillnastr

import numpy as np
np.random.seed(0)  # for reproducibility


In [ ]:
# Nice plots!
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# PARAMETERS

# Unified database, not yet postprocessed
unified_csv = r'databases_output\merged_fmp_steph_manon_sarah_dicom_ecg_reports_unifiedall_nifti.csv'
unifiedpersubj_csv = r'databases_output\merged_fmp_steph_manon_sarah_dicom_ecg_reports_unifiedall.csv'
output_dir = r'databases_output'

# Hide null values in plots?
plot_hide_nan = True

diagorder_doc = ['', 'na', 'impossible', 'braindead', 'coma', 'vs/uws', 'mcs', 'mcs-', 'mcs+', 'srmcs', 'emcs', 'lis', 'lis_incomplete', 'partial lis']


---------------
## PREPARE DATASET (AND ONLYDOC DATASET)

In [ ]:
# Import the csv dbs as dataframes
import pandas as pd
import numpy as np

cf_unified = pd.read_csv(unified_csv, sep=';', low_memory=False).dropna(axis=0, how='all').fillna('')  # drop empty lines
cf_unified

In [ ]:
cf_unified['unified.diagnosis_best'].unique()

In [ ]:
# filter to keep only doc patients (susceptible to being sedated)
cf_unified_onlydoc = cf_unified[cf_unified['unified.diagnosis_best'].isin(['vs/uws', 'mcs', 'mcs+', 'mcs-', 'emcs', 'srmcs', 'coma', 'lis', 'lis_incomplete', 'partial lis', 'conflict', 'braindead'])]


In [ ]:
# Group by name
cf_unified_onlydoc_byname = cf_unified_onlydoc.groupby('name').agg(concat_vals_unique)

In [ ]:
# Check diagnoses count is fine (sanity check)
cf_unified_onlydoc_byname.reset_index().loc[:, ['name', 'unified.diagnoses_count']]

In [ ]:
save_df_as_csv(cf_unified_onlydoc_byname, 'onlydoc.csv', fields_order=False)

----------------------
## FOR MURIELLE (MRI SEDATION STATS)

In [ ]:
with open('bynamecounts.txt', 'w') as f:
    f.write(cf_unified_onlydoc_byname.count().to_string())
cf_unified_onlydoc_byname.count()

In [ ]:
cf_unified_onlydoc[cf_unified_onlydoc['nifti.func OK'].isin(['O', 'M', 'M2', 'N'])].groupby('name').agg(concat_vals_unique).count()

In [ ]:
cf_unified_onlydoc[cf_unified_onlydoc['nifti.struct OK (for fmri)'].isin(['O', 'M', 'M2', 'N', 'W'])].groupby('name').agg(concat_vals_unique).count()

In [ ]:
# Agregate per MRI sessions
cf_unified_onlydoc_sess = cf_unified_onlydoc[~cf_unified_onlydoc['StudyDate'].isnull() & (cf_unified_onlydoc['StudyDate'] != '')].groupby(['name', 'StudyDate']).agg(concat_vals_unique)
cf_unified_onlydoc_sess

In [ ]:
cf_unified_onlydoc_sess[~cf_unified_onlydoc_sess['nifti.func OK'].isin(['X', ''])].count()

In [ ]:
cf_unified_onlydoc_sess[~cf_unified_onlydoc_sess['nifti.struct OK (for fmri)'].isin(['X', ''])].count()

In [ ]:
def saveepisedat(cf, appendtext=''):
    a = cf['unified.episedation']
    b = a.astype('str').value_counts()
    c = b.to_frame().reset_index().rename(columns={'index': 'sedation', 'unified.episedation': 'count'})
    #df_to_unicode_fast(c).to_excel(unified_csv[:-4] + '_episedationcount%s.xls' % appendtext)  # for python 2
    c.to_excel(unified_csv[:-4] + '_episedationcount%s.xls' % appendtext)
    return True
saveepisedat(cf_unified_onlydoc_sess, '_persess')
saveepisedat(cf_unified_onlydoc_byname, '_persubject')

In [ ]:
fig = plt.figure()
#toplot = cf_unified_perdiag[cf_unified_perdiag['unified.diagnosis_worst'] == diag]['unified.diagnosis_best'].astype('str').value_counts(dropna=plot_hide_nan)
cf_unified_onlydoc_byname['unified.etiology'].value_counts().plot(fig=fig, kind='pie', title='Etiology of DOC patients\n%i patients' % (cf_unified_onlydoc_byname.shape[0]), autopct='%.1f%%', figsize=(15,15))
plt.axis('off')
fig.savefig(os.path.join(output_dir, 'fig_docetio.png'), bbox_inches='tight', dpi=600)
with open(os.path.join(output_dir, 'fig_docetio.txt'), 'w') as f:
    f.write(cf_unified_onlydoc_byname['unified.etiology'].value_counts().to_string())

In [ ]:
import codecs
cf_unified_onlydoc_sess.loc[cf_unified_onlydoc_sess['unified.diagnosis_best'] == 'srmcs', 'unified.diagnosis_best'] = 'mcs+'
for diag in cf_unified_onlydoc_sess['unified.diagnosis_best'].unique():
    fig = plt.figure()
    toplot = cf_unified_onlydoc_sess.loc[cf_unified_onlydoc_sess['unified.diagnosis_best'] == diag, 'unified.episedation']
    toplot.value_counts().plot(fig=fig, kind='pie', title='Sedation for diag %s\n%i sessions' % (diag.replace('/', '-'), toplot.shape[0]), autopct='%.1f%%', figsize=(15,15))
    plt.axis('off')
    fig.savefig(os.path.join(output_dir, 'fig_sedat_%s.png' % diag.replace('/', '-')), bbox_inches='tight', dpi=600)
    with codecs.open(os.path.join(output_dir, 'fig_sedat_%s.txt' % diag.replace('/', '-')), 'w', 'utf-8-sig') as f:
        f.write(toplot.to_string())

--------------------
## MARKOV CHAIN

In [ ]:
# Import the csv dbs as dataframes
import pandas as pd
import numpy as np

cf_unifiedsubj = pd.read_csv(unifiedpersubj_csv, sep=';', low_memory=False).dropna(axis=0, how='all').fillna('')  # drop empty lines
cf_unifiedsubj

In [ ]:
cf_unifiedsubj['unified.diagnosis_best'].unique()

In [ ]:
# filter to keep only doc patients (susceptible to being sedated)
cf_unifiedsubj_onlydoc = cf_unifiedsubj[cf_unifiedsubj['unified.diagnosis_best'].isin(['vs/uws', 'mcs', 'mcs+', 'mcs-', 'emcs', 'srmcs', 'coma', 'lis', 'lis_incomplete', 'partial lis', 'conflict', 'braindead'])]
cf_unifiedsubj_onlydoc

In [ ]:
cf_unified_onlydoc_byname

In [ ]:
cf_unified_onlydoc_byname.reset_index().loc[:, ['name', 'unified.diagnoses_count']]

In [ ]:
find_columns_matching(cf_unified_onlydoc_byname, ['count'])

In [ ]:
# Extract max crsr count
cf_unified_onlydoc_byname['unified.diagnoses_count_withdate'] = cf_unified_onlydoc_byname['unified.diagnoses_count_withdate'].apply(lambda x: max(x) if isinstance(x, list) else x)
cf_unified_onlydoc_byname['unified.diagnoses_count_withdate']

In [ ]:
diagcount = df_squash_lists(cf_unified_onlydoc_byname['unified.diagnoses_count'], func=max, aggressive=True)
diagcount.astype('int')

In [ ]:
# Show number of patients with at least 1, then 2 CRS-R but not necessarily with date
diagcount = df_squash_lists(cf_unified_onlydoc_byname['unified.diagnoses_count'], func=max, aggressive=True).astype('int')
diagcountwithdate = df_squash_lists(cf_unified_onlydoc_byname['unified.diagnoses_count_withdate'], func=max, aggressive=True).astype('float')
print('At least one CRS-R but not necessarily with dates:')
print((diagcount >= 1).sum())
print('At least two CRS-R but not necessarily with dates:')
print((diagcount >= 2).sum())
print('At least one CRS-R with date:')
print((diagcountwithdate >= 1).sum())
print('At least two CRS-R with dates:')
print((diagcountwithdate >= 2).sum())
print('Both worst and best diags have dates:')
print(((~cf_unified_onlydoc_byname['unified.diagnoses_firstbestdiag'].isnull()) & (~cf_unified_onlydoc_byname['unified.diagnoses_firstworstdiag'].isnull()) & (diagcountwithdate >= 2)).sum())
print('Both worst and best diags have dates OR best diag is LIS or lis incomplete:')
doc2diagdatepluslis = \
    (
    # at least 2 diagnoses with both best diag and worst diag having dates info
    ((~cf_unified_onlydoc_byname['unified.diagnoses_firstbestdiag'].isnull()) & (~cf_unified_onlydoc_byname['unified.diagnoses_firstworstdiag'].isnull()) & (diagcountwithdate >= 2))
    # or it's a LIS, then no need for dates (because anyway there's none, there is no CRS-R)
    | (df_squash_lists(cf_unified_onlydoc_byname['unified.diagnosis_best'], aggressive=True).isin(['lis', 'lis_incomplete']))
    )
print(doc2diagdatepluslis.sum())

In [ ]:
# Find patients where the best diagnosis is a list (should not happen but it happens ???)
cf_unified_onlydoc_byname.loc[cf_unified_onlydoc_byname['unified.diagnosis_best'].apply(lambda x: isinstance(x, list)), 'unified.diagnosis_best']

In [ ]:
# Select only patients with both best and worst diagnoses with dates (else can't see any transition)
cf_unified_onlydoc_byname_min2diag = cf_unified_onlydoc_byname.loc[doc2diagdatepluslis, :]
# Also get all patients with at least two CRS-Rs but not necessarily with dates. By nature this is less precise as we can't know the direction (improvement, fluctuation or worsening) if we don't have the dates
cf_unified_onlydoc_byname_min2diagunprecise = cf_unified_onlydoc_byname.loc[diagcount >= 2, :]
# Drop the 'test test' patient
cf_unified_onlydoc_byname_min2diag.drop('test test', inplace=True)
cf_unified_onlydoc_byname_min2diagunprecise.drop('test test', inplace=True)
# Show
cf_unified_onlydoc_byname_min2diag

In [ ]:
# Filter out false patients names
falsepatientsnames = [
    'prise en charge',
    'ant la',
    'association',
    'locked in',
    'consciousness',
    'cerebral',
    'references',
    'qualite de',
    'durees',
    'enfants et les',
    'somatosensory',
    'such reposiiories',
    'evaluation',
    'regressing',
    'locked-in',
    'two millennia',
    'neuroimagerie',
    'prognosis',
    'traumatic',
    'italie et',
    'a completer',
    'acute and subacute',
    'ant les ',
    'defined by',
    'diagnosis'
    'ant votre ',
    'ant son ',
    'ant leurs ',
    ]
cf_unified_onlydoc_byname_min2diag.drop(cf_unified_onlydoc_byname_min2diag.index[cf_unified_onlydoc_byname_min2diag.index.str.contains(r'(?:{})'.format('|'.join(falsepatientsnames)))], inplace=True)
cf_unified_onlydoc_byname_min2diag

In [ ]:
# Convert dates count to an int, will be easier to process (and cleaner figures)
cf_unified_onlydoc_byname_min2diag.loc[cf_unified_onlydoc_byname_min2diag['unified.diagnoses_count_withdate'].isnull(), 'unified.diagnoses_count_withdate'] = 0  # replace nan values
cf_unified_onlydoc_byname_min2diag.loc[:, 'unified.diagnoses_count_withdate'] = cf_unified_onlydoc_byname_min2diag['unified.diagnoses_count_withdate'].astype('int')
cf_unified_onlydoc_byname_min2diagunprecise.loc[:, 'unified.diagnoses_count_withdate'] = cf_unified_onlydoc_byname_min2diagunprecise['unified.diagnoses_count_withdate'].astype('int')
cf_unified_onlydoc_byname_min2diag['unified.diagnoses_count_withdate']

In [ ]:
cf_unified_onlydoc_byname_min2diag['unified.diagnoses_count_withdate'].plot(kind='hist', bins=max(cf_unified_onlydoc_byname_min2diag['unified.diagnoses_count_withdate']))

In [ ]:
# Show cases where there are multiple best or worst diagnoses (which should not happen)
conflictdiags = cf_unified_onlydoc_byname_min2diag.loc[cf_unified_onlydoc_byname_min2diag['unified.diagnosis_worst'].apply(lambda x: isinstance(x, list)), :].index
cf_unified_onlydoc_byname_min2diag.loc[conflictdiags, find_columns_matching(cf_unified_onlydoc_byname_min2diag, 'unified')]

In [ ]:
# Fix cases where there are multiple best/worst diagnoses, by selecting the best/worst diagnosis respectively

# Order diagnoses using Pandas discrete categories, so that we can easily grade the maximum and minimum diagnoses
cf_unified_onlydoc_byname_min2diag.loc[:, 'unified.diagnosis_worst'] = cf_unified_onlydoc_byname_min2diag['unified.diagnosis_worst'].apply(lambda x: compute_best_diag(x, diag_order=diagorder_doc, persubject=None).min() if not isinstance(x, str) and x is not None else x)
cf_unified_onlydoc_byname_min2diag.loc[:, 'unified.diagnosis_best'] = cf_unified_onlydoc_byname_min2diag['unified.diagnosis_best'].apply(lambda x: compute_best_diag(x, diag_order=diagorder_doc, persubject=None).max() if not isinstance(x, str) and x is not None else x)
# Same for unprecise dataframe
cf_unified_onlydoc_byname_min2diagunprecise.loc[:, 'unified.diagnosis_worst'] = cf_unified_onlydoc_byname_min2diagunprecise['unified.diagnosis_worst'].apply(lambda x: compute_best_diag(x, diag_order=diagorder_doc, persubject=None).min() if not isinstance(x, str) and x is not None else x)
cf_unified_onlydoc_byname_min2diagunprecise.loc[:, 'unified.diagnosis_best'] = cf_unified_onlydoc_byname_min2diagunprecise['unified.diagnosis_best'].apply(lambda x: compute_best_diag(x, diag_order=diagorder_doc, persubject=None).max() if not isinstance(x, str) and x is not None else x)


In [ ]:
# Sanity check if the previous docs with conflicting diagnoses are now ok
cf_unified_onlydoc_byname_min2diag.loc[conflictdiags, find_columns_matching(cf_unified_onlydoc_byname_min2diag, 'unified')]

In [ ]:
#a = cf_unified_onlydoc_byname_min2diag.loc[cf_unified_onlydoc_byname_min2diag['unified.diagnosis_worst'].apply(lambda x: isinstance(x, list)), find_columns_matching(cf_unified_onlydoc_byname_min2diag, 'unified')]
# correct:
#print(a['unified.diagnosis_worst'].apply(lambda x: compute_best_diag(x, diag_order=['', 'na', 'impossible'] + diagorder_doc + ['lis'], persubject=None).min()))
#print(a['unified.diagnosis_worst'].apply(lambda x: compute_best_diag(x, diag_order=['', 'na', 'impossible'] + diagorder_doc + ['lis'], persubject=None).max()))
# wrong:
#print(a['unified.diagnosis_worst'].apply(lambda x: min(compute_best_diag(x, diag_order=['', 'na', 'impossible'] + diagorder_doc + ['lis'], persubject=None)))
#print(a['unified.diagnosis_worst'].apply(lambda x: max(compute_best_diag(x, diag_order=['', 'na', 'impossible'] + diagorder_doc + ['lis'], persubject=None)))

In [ ]:
#valsorder = ['AAA', 'BBBBBBBBB', 'CCCCC', 'DD', 'EEE']
#s = pd.Series(valsorder[1:4])
#s = s.astype(pd.api.types.CategoricalDtype(categories=valsorder, ordered=True))
#min(s)

In [ ]:
# Without dates transition matrix (so that we can't know in what direction the transition happened), but like this we have also non-doc such as lis, which don't have CRS-R (and no dates), only the diagnosis
def calc_transition_matrix(df, col1, col2):
    """Compute the transition matrix between two columns containing categorical values (but not necessarily CategoricalDTypes) in a pandas DataFrame.
    Returns two dataframes: the first with probabilities, the second with counts"""
    try:
        tmat = pd.DataFrame(0, index=df[col1].unique(), columns=df[col2].unique())
    except TypeError as exc:
        tmat = pd.DataFrame(0, index=df[col1].astype('str').unique(), columns=df[col2].astype('str').unique())

    for idx, row in df.iterrows():
        # If worst diag is empty, then it's probably a LIS (no CRS-R, only one diagnosis), then we take it as a no change
        if row[col1] is None or row[col1] == '':
            tmat.loc[row[col2], row[col2]] += 1
        else:
            tmat.loc[row[col1], row[col2]] += 1
    
    # Compute probabilities
    tmatcount = tmat.copy()
    tmat = tmat.apply(lambda x: x / x.sum(), axis=1)
    # Return results
    return tmat, tmatcount

tmat, tmatcount = calc_transition_matrix(cf_unified_onlydoc_byname_min2diag, 'unified.diagnosis_worst', 'unified.diagnosis_best')
tmatu, tmatucount = calc_transition_matrix(cf_unified_onlydoc_byname_min2diagunprecise, 'unified.diagnosis_worst', 'unified.diagnosis_best')
tmat

In [ ]:
tmatcount

In [ ]:
plt.matshow(tmat)

In [ ]:
# Reorder columns and indices
def reordertransitionmatrix(df, orderlist):
    df2 = df.copy()
    df2 = df2.loc[:, [x for x in orderlist if x in df2.columns]]  # easiest way: get the whole ordered list and filter it through the existing columns
    df2 = df2.loc[[x for x in orderlist if x in df2.index], :]
    return df2

tmat = reordertransitionmatrix(tmat, diagorder_doc)
tmatcount = reordertransitionmatrix(tmatcount, diagorder_doc)
tmat

In [ ]:
print(tmatcount.sum().sum())
tmatcount

In [ ]:
# Without ensuring that both dates are present, we just know that there are at least 2 CRS-R diagnoses
# There are 200 more subjects like this, but not sure they are all correct (some diagnoses might have been duplicated)
# TODO: add back the LIS also in this dataset if I use it!
tmatu = reordertransitionmatrix(tmatu, diagorder_doc)
tmatucount = reordertransitionmatrix(tmatucount, diagorder_doc)
print(tmatucount.sum().sum())
tmatucount

In [ ]:
def plotheatmap(df):
    df2 = df.copy()  # make a copy of the dataframe, avoids side effects
    df2[df2==0] = float('NaN')  # make 0 values blank
    plt.pcolor(df2, cmap=plt.get_cmap('viridis'))
    plt.yticks(np.arange(0.5, len(df2.index), 1), df2.index)
    plt.xticks(np.arange(0.5, len(df2.columns), 1), df2.columns)
    plt.show()

plotheatmap(tmat)

In [ ]:
# Transition matrix for at least 5 CRS-R (with at least worst and best with dates info)
tmat5, tmat5count = calc_transition_matrix(cf_unified_onlydoc_byname_min2diag.loc[cf_unified_onlydoc_byname_min2diag['unified.diagnoses_count_withdate'] >= 5, :], 'unified.diagnosis_worst', 'unified.diagnosis_best')
tmat5 = reordertransitionmatrix(tmat5, diagorder_doc)
tmat5count = reordertransitionmatrix(tmat5count, diagorder_doc)
print(tmat5)
print(tmat5count)
plotheatmap(tmat5)

In [ ]:
# RESULT: compare tmat5 (at least 5 CRS-R, down) to tmat (at least 2 CRS-Rs, up figure), it suggests that mcs- might be most of the time a misdiagnosis of mcs+. Same for mcs+, misdiagnosis of emcs. srmcs always changes to emcs, so the criterion requiring 2 consecutive is not necessary, but we have a small sample. This suggests modifications to guidelines (READ REF giacino guidelines).
plotheatmap(tmat)
plotheatmap(tmat5)
print(tmat)
print(tmat5)

In [ ]:
def compute_transition_matrix_totals(df, colsnames=None):
    '''Compute total sum over each row of a square matrix: sum of what is left to current index, what is on self node, and sum on the right'''
    res = pd.DataFrame(index=df.index, columns=['left', 'self', 'right'])
    for idx, row in df.iterrows():
        res.loc[idx, 'left'] = row[:row.index.get_loc(idx)].sum()
        if row.index.get_loc(idx) < len(row.index):
            res.loc[idx, 'right'] = row[row.index.get_loc(idx)+1:].sum()
        res.loc[idx, 'self'] = row[idx]
    # Rename columns if supplied
    if colsnames is not None and len(colsnames) == 3:
        res.columns = colsnames
    # Return
    return res

def calc_transition_matrix_withdates(df_in, diagorder, col1, col2, col1datefirst, col2datefirst, col1datelast, col2datelast):
    """Compute the transition matrix between two columns containing categorical values (but not necessarily CategoricalDTypes) in a pandas DataFrame.
    This takes into account the dates, so that the transition matrix will be directional. This also allows to compute the totals.
    col1 is the initial state, col2 the resulting state.
    Returns two dataframes: the first with probabilities, the second with counts"""

    # Copy to avoid side effects
    df = df_in.copy()

    # Prepare the input columns to date format
    df.loc[:, col1datefirst] = convert_to_datetype(df_squash_lists(df[col1datefirst], lambda x: x[0]), col1datefirst, '%Y-%m-%d').set_index(df.index)
    df.loc[:, col2datefirst] = convert_to_datetype(df_squash_lists(df[col2datefirst], lambda x: x[0]), col2datefirst, '%Y-%m-%d').set_index(df.index)
    df.loc[:, col1datelast] = convert_to_datetype(df_squash_lists(df[col1datelast], lambda x: x[-1]), col1datelast, '%Y-%m-%d').set_index(df.index)
    df.loc[:, col2datelast] = convert_to_datetype(df_squash_lists(df[col2datelast], lambda x: x[-1]), col2datelast, '%Y-%m-%d').set_index(df.index)

    # Initialize the transition matrix with the appropriate columns and indices
    try:
        tmat = pd.DataFrame(0, index=df[col1].unique().tolist(), columns=df[col2].unique().tolist())
    except TypeError as exc:
        tmat = pd.DataFrame(0, index=df[col1].astype('str').unique().tolist(), columns=df[col2].astype('str').unique().tolist())

    # Reorder the columns/indices (the order is important to compute the totals later)
    #tmat = reordertransitionmatrix(tmat, diagorder+totalcols)
    tmat = reordertransitionmatrix(tmat, diagorder)

    # Initialize the fluctuation matrix
    tmatfluc = tmat.copy()

    # Build the transition matrix by incrementing counters
    for idx, row in df.iterrows():
        # If worst diag is empty, then it's probably a LIS (no CRS-R, only one diagnosis), then we take it as a no change
        if row[col1] is None or row[col1] == '':
            tmat.loc[row[col2], row[col2]] += 1
        # Else it's the same diagnosis for worst and best, this is stable, we simply add it
        elif row[col1] == row[col2]:
            tmat.loc[row[col1], row[col2]] += 1
        # Else both diagnoses are different, we need to know if it's an improvement or a worsening or even a fluctuation, we will use the date info to disambiguate
        else:
            # Calculate delay between the last worst diagnosis and the last best diagnosis
            deltalastworst2lastbest = (row[col2datelast] - row[col1datelast]).days
            # Switch to appropriate direction depending on the value of this delay
            if deltalastworst2lastbest > 0:
                # Positive delay: last worst diagnosis -> last best diagnosis: the last best diagnosis happened later than the last worst diagnosis, apriori it's an improvement...
                # ...but just in case, check if maybe the best diagnosis also appeared first before the last worst diagnosis, which would mean that it's fluctuating between both
                deltafirstbest2lastworst = (row[col1datelast] - row[col2datefirst]).days
                if deltafirstbest2lastworst <= 0:
                    # Negative or null delay: the first best diagnosis happened after the last worst diagnosis, so this confirms that it's NOT a fluctuation but an improvement
                    tmat.loc[row[col1], row[col2]] += 1
                else:
                    # Positive delay: this confirms this is a fluctuation: there was a temporal sequence of best diagnosis -> worst diagnosis -> best diagnosis, so that's clearly a fluctuation
                    tmatfluc.loc[row[col1], row[col2]] += 1
            elif deltalastworst2lastbest < 0:
                # Negative delay: last best diagnosis -> last worst diagnosis. So this is a worsening apriori...
                # ...but we also check if maybe there was a sequence like worst diagnosis -> best diagnosis -> worst diagnosis, indicative of a fluctuation.
                deltafirstworst2lastbest = (row[col2datelast] - row[col1datefirst]).days
                if deltafirstworst2lastbest <= 0:
                    # Negative or null delay: this confirms this is a worsening, not a fluctuation
                    tmat.loc[row[col2], row[col1]] += 1
                else:
                    # Positive delay: there is indeed a worst diagnosis before the last best diagnosis, so this is a fluctuation
                    tmatfluc.loc[row[col1], row[col2]] += 1
            elif deltalastworst2lastbest == 0:
                # Null delay: both the last best and last worst diagnoses happened on the same day, so we can't know for sure what happened (we don't have the exact timing, the day is the highest resolution we have), so we assume a fluctuation (just to be same)
                tmatfluc.loc[row[col1], row[col2]] += 1
    
    # Compute totals
    totalcols = ['worsening', 'nochange', 'improvement']
    tmattot = compute_transition_matrix_totals(tmat, totalcols)
    tmatfluctot = compute_transition_matrix_totals(tmatfluc, totalcols)

    # Return results
    return tmat, tmattot, tmatfluc, tmatfluctot

def transition_matrix_to_proba(tmat):
    '''Compute probabilities from a transition matrix containing the count occurrences of each transitions'''
    return tmatproba.apply(lambda x: x / x.sum(), axis=1)

calc_transition_matrix_withdates(cf_unified_onlydoc_byname_min2diag, diagorder_doc,
                                 'unified.diagnosis_worst', 'unified.diagnosis_best',
                                 'unified.diagnoses_firstworstdiag', 'unified.diagnoses_firstbestdiag',
                                 'unified.diagnoses_lastworstdiag', 'unified.diagnoses_lastbestdiag')

In [ ]:
# LIMITATIONS OF THIS STUDY:
# * does not account for temporality between worst and best diagnosis, thus worst diagnosis may well be an evolution happening later than the best diagnosis. Here we show the possible transitions between both, should be considered bidirectional. Thus interpretation is not necessarily of an evolution but a possible transition between both states.
# we could change that but what criterion should we use? And what timeframe, if it's a daytoday fluctuation, should we consider this is ...? Or simply restrict analysis to all crs-r timeframe under 3 months, so we consider it's not evolution, only fluctuation or short term evolution.

# RESULTS
#* most change diag, dont be fooled by the heatmap, so this and graph are bad viz, they dont show the main result. Problem with heatmap is the colors: how do you add the colors to know that in fact where it's most salient isn't the majority of the changes?
#* SOLUTION: add 3 columns: worsening, no change and improvement, and these will be the sum of enhancement vs no change vs worsening. Simple to calculate: same position in x and y = no change, below position in columns compared to index = worsening, opposite is improvement.
#* srmcs 50% chance change to emcs. we question the pertinence of requiring 2 consecutive fulfillment of the tasks

# TODO:
#* account for bidirectionality by detecting order of worst and best diag?

In [ ]:
os.environ["PATH"]

In [ ]:
from __future__ import division  # Only for how I'm writing the transition matrix
import networkx as nx  # For the magic
import matplotlib.pyplot as plt  # For plotting

# Install pydot and graphviz beforehand, and change the path below on Windows to your graphviz folder

# Add graphviz to the path, so that the binaries can be found (if you get an error about dot not being found, that's why, modify this path here AND also you need nxpydot and pydotplus installed)
os.environ["PATH"] += os.pathsep + r'C:/Program Files (x86)/Graphviz2.38/bin'

# and the following code block is not needed
# but we want to see which module is used and
# if and why it fails
try:
    import pydotplus # also requires nxpydot installed but no need to import
    from networkx.drawing.nx_pydot import write_dot, to_pydot
    print("using package pydotplus")
except ImportError:
    print()
    print("Module pydotplus and nxpydot were not found (nxpydot is a wrapper for networkx to use pydotplus, please uninstall pydot beforehand as it is incompatible with python 3) ")
    print("see https://networkx.github.io/documentation/latest/reference/drawing.html")
    print()
    raise

def transition_to_graph(df):
    # Adapted from https://vknight.org/unpeudemath/code/2015/11/15/Visualising-markov-chains.html
    G = nx.MultiDiGraph(directed=True)
    labels={}
    edge_labels={}

    for state1 in df.index:
        for state2 in df.columns:
            weight = df.loc[state1, state2]
            if weight > 0:
                G.add_edge(state1,
                           state2,
                           weight=weight,
                           penwidth=weight*10,
                           label="{:.02f}".format(weight))
                edge_labels[(state1, state2)] = label="{:.02f}".format(weight)
    return G

def plot_transition_graph(G, pos=None):
    # Plot using networkx internal visualization with matplotlib, it does not support multiple directed edges (use graphviz instead)
    # https://stackoverflow.com/questions/20133479/how-to-draw-directed-graphs-using-networkx-in-python
    plt.figure(figsize=(14,7))
    #node_size = 200
    #pos = {state:list(state) for state in states}
    #nx.draw_networkx_edges(G,pos,width=1.0,alpha=0.5)
    #nx.draw_networkx_labels(G, pos, font_weight=2)
    options = {
        'node_color': 'cyan',
        'node_size': 2000,
        'width': 1,
        'arrowstyle': '-|>',
        'arrowsize': 30,
    }
    if pos is None:
        # Get the layout defined manually in G
        pos = nx.get_node_attributes(G,'pos')
        if not pos:
            # Else calculate a layout automatically
            #pos = nx.nx_pydot.graphviz_layout(G, prog='neato')
            pos = nx.drawing.layout.spectral_layout(G)
    nx.draw(G, pos, arrows=True, with_labels=True, **options)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.axis('off')

def plot_transition_graph2(G, pos=None):
    # In-memory plotting using GraphViz (supports multiple directed edges between two nodes), pydotplus and nxpydot via networkx
    from io import BytesIO
    import matplotlib.image as mpimg
    # add graphviz layout options (see https://stackoverflow.com/a/39662097)
    G.graph['edge'] = {'arrowsize': '0.6', 'splines': 'curved', 'rankdir':'LR'}
    G.graph['graph'] = {'scale': '3'}

    # adding attributes to edges in multigraphs is more complicated but see
    # https://stackoverflow.com/a/26694158
    #G[1][1][0]['color']='red'

    # From: https://stackoverflow.com/questions/4596962/display-graph-without-saving-using-pydot
    # convert from networkx -> pydot
    pydot_graph = to_pydot(G)

    # render pydot by calling dot, no file saved to disk
    png_str = pydot_graph.create_png(prog='dot') # can change to dot or twopi, but not neato because the latter is only for non directed graphs

    # treat the dot output string as an image file
    sio = BytesIO()
    sio.write(png_str)
    sio.seek(0)
    img = mpimg.imread(sio)

    # plot the image
    plt.figure(figsize=(40,15))
    imgplot = plt.imshow(img, aspect='equal')
    plt.axis('off')
    plt.show(block=False)

G = transition_to_graph(tmat)
# set position manually
#for i, n in enumerate(G):
#    G.node[n]['pos'] = '"%d,%d"' % (i, 1)
write_dot(G, 'mc.dot')
#plot_transition_graph2(G) # TODO: fix this (or just use next cell?), it now fails but dunno why

In [ ]:
# In-memory plotting with better settings
from io import BytesIO
import matplotlib.image as mpimg

# convert from networkx -> pydot
pydot_graph = to_pydot(G)
pydot_graph.set_concentrate(True)
pydot_graph.set_layout('dot')
pydot_graph.set_dpi(300)
pydot_graph.set_pack(True)
#pydot_graph.set_rank('same')
pydot_graph.set_splines('line')

# render pydot by calling dot, no file saved to disk
png_str = pydot_graph.create_png(prog='dot') # can change to dot or twopi, but not neato because the latter is only for non directed graphs

# treat the dot output string as an image file
sio = BytesIO()
sio.write(png_str)
sio.seek(0)
img = mpimg.imread(sio)

# plot the image
plt.figure(figsize=(40,15))
imgplot = plt.imshow(img, aspect='equal')
plt.axis('off')
plt.show(block=False)

In [ ]:
cf_unified_onlydoc_byname_min2diag['unified.diagnosis_best'].astype('str').unique()

In [ ]:
find_columns_matching(cf_unified_onlydoc_byname_min2diag, 'unified')

---------------------------------
## Predictive model with machine learning

### Data preprocessing

In [ ]:
# Extract the columns we want to build the predictive model
mlcols = ['unified.diagnosis_best', 'unified.diagnosis_worst', 'unified.diagnoses_timeperiod',
          'unified.diagnoses_count', 'unified.age', 'unified.gender', 'unified.acute', 'unified.etiology']

cf_ml = cf_unified_onlydoc_byname_min2diag.reset_index().loc[:, mlcols]
cf_ml

In [ ]:
# Squash lists so that we have only one value per feature per sample
from collections import Counter

def calcmode(nparr):
    # From: https://stackoverflow.com/questions/16330831/most-efficient-way-to-find-mode-in-numpy-array
    # There is also a multi-dimensional version available
    # in worst case, return the min
    mode = Counter(nparr).most_common(1)
    # mode will be [(6,3)] to give the count of the most occurring value, so ->
    m = mode[0][0]
    return m

cf_ml = cf_ml.apply(lambda x: df_squash_lists(x, func=calcmode, aggressive=True))
cf_ml

In [ ]:
# Convert timeperiod to ints (representing the days) and then a logarithm transform
# Alternatives: https://stackoverflow.com/questions/9775743/how-can-i-parse-free-text-time-intervals-in-python-ranging-from-years-to-second
#datetime.strptime(cf_ml['unified.diagnoses_timeperiod'][0], '%d days %H:%M:%S.%f000').day

# Get the number of days from the timedelta string
cf_ml['unified.diagnoses_timeperiod'] = cf_ml['unified.diagnoses_timeperiod'].str.extract('\s*(\d+)\s+').astype('float')
# Convert to logarithm
cf_ml['unified.diagnoses_timeperiod'] = cf_ml['unified.diagnoses_timeperiod'].apply(lambda x: np.log(x))
# Since we use log, 0 will become -inf, so we restore back the 0 value for those (which means that all CRS-R assessments happened on the same day)
cf_ml.loc[cf_ml['unified.diagnoses_timeperiod'] == float('-inf'), 'unified.diagnoses_timeperiod'] = 0.0
# Show result
cf_ml['unified.diagnoses_timeperiod']

In [ ]:
# Clean up age and convert to float (so that it's not a categorical feature but an ordinal, with an order)
cf_ml.loc[cf_ml['unified.age'].isnull() | (cf_ml['unified.age'] == 'None') | (cf_ml['unified.age'] == '#VALUE!'), 'unified.age'] = float('NaN')
cf_ml['unified.age'] = cf_ml['unified.age'].astype('float')
cf_ml['unified.age']

In [ ]:
# Check which subjects have very old age
# You'll have to fix them manually if there are errors
agecheck = df_fillnastr(df_squash_lists(cf_unified_onlydoc_byname_min2diag['unified.age'], func=calcmode, aggressive=True), float('NaN')).astype('float')
cf_unified_onlydoc_byname_min2diag.loc[(agecheck < 15) | (agecheck > 90), ['unified.age'] + find_columns_matching(cf_unified_onlydoc_byname_min2diag, 'unified')]


In [ ]:
cf_ml[(cf_ml['unified.age'] < 15) | (cf_ml['unified.age'] > 90)]

In [ ]:
# Fix manually age issues
cf_ml.loc[152, 'unified.age'] = 33.0
cf_ml.loc[250, 'unified.age'] = 22.0
cf_ml.loc[410, 'unified.age'] = 31.0
cf_ml.loc[2, 'unified.age'] = 57.0
cf_ml.loc[120, 'unified.age'] = 38.0
cf_ml.loc[125, 'unified.age'] = 42.0
cf_ml.loc[175, 'unified.age'] = 24.0

# Check they disapper
cf_ml[(cf_ml['unified.age'] < 15) | (cf_ml['unified.age'] > 90)]

In [ ]:
cf_ml['unified.etiology'].value_counts()

In [ ]:
# Convert diagnoses count to int type
# Note: use df_fillnastr in case of issues with nan values
cf_ml['unified.diagnoses_count'] = cf_ml['unified.diagnoses_count'].astype('int')
cf_ml['unified.diagnoses_count']

### Missing data plot

In [ ]:
import missingno as msno

In [ ]:
for col in cf_ml.columns:
    print('= Values for column %s (first without and then with log scale):' % col)
    if str(cf_ml[col].dtype).startswith('float') or str(cf_ml[col].dtype).startswith('int'):
        # Without log
        plt.hist(cf_ml[col], range=[cf_ml[col].min(), cf_ml[col].max()], bins=20)
        plt.show()
        # With log
        plt.hist(cf_ml[col], range=[cf_ml[col].min(), cf_ml[col].max()], bins=20, log=True)
        plt.show()
    else:
        print(cf_ml[col].value_counts())
    print('\n')

In [ ]:
# Convert null strings in all str and object to real None object so that missingno can detect them
def fillfakena(df_in):
    '''Convert null strings in all str and object to real None object so that missingno can detect them'''
    df = df_in.copy()
    for col in df.columns:
        if df[col].dtype == 'str' or df[col].dtype == 'object' or df[col].dtype.kind == 'O':
            df[col] = df_fillnastr(df[col], None)
    return df

cf_ml = fillfakena(cf_ml)

In [ ]:
msno.matrix(cf_ml)

In [ ]:
msno.bar(cf_ml)

In [ ]:
# No real pairwise missingness correlation, only shows that for patients where more infos filled, the data is more complete. So when the operator took the time to do a proper job.
msno.heatmap(cf_ml)

In [ ]:
# age and acute not so much related here, it's the last merge before final
msno.dendrogram(cf_ml)

### Data preparation for machine learning

In [ ]:
# Convert back null values to null strings, making them their own class (separate class strategy for handling missing values)
def fillnaseparateclass(df_in):
    '''Convert back null values to null strings, making them their own class (separate class strategy for handling missing values)'''
    df = df_in.copy()
    for col in df.columns:
        if df[col].dtype == 'str' or df[col].dtype == 'object' or df[col].dtype.kind == 'O':
            df.loc[df[col].isnull(), col] = 'None'
    return df

cf_ml = fillnaseparateclass(cf_ml)

In [ ]:
# Prepare dataset by separating y class from X features
X_orig = cf_ml.drop(columns='unified.diagnosis_best')
y_orig = cf_ml['unified.diagnosis_best']

In [ ]:
# Get the list of categorical columns
categorical_columns = X_orig.select_dtypes(exclude=['int','float']).columns
categorical_columns

### Data splitting (test dataset, cross-validation dataset)

In [ ]:
# Split to get a test set, and separate X from y (target class) but by balancing to keep same proportions in both relatively to y
# Although balance is not strictly guaranted: https://github.com/scikit-learn/scikit-learn/issues/8913
from sklearn import model_selection

splitter = model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.1, train_size=None, random_state=0)
train_idx, test_idx = next(splitter.split(X_orig, y_orig))
X_train, X_test = X_orig.loc[train_idx, :], X_orig.loc[test_idx, :]
y_train, y_test = y_orig[train_idx], y_orig[test_idx]

### Data augmentation - balanced learning

In [ ]:
# Oversample randomly (duplicating rows as-is) to balance classes
oversample = True

def random_oversample(X, y):
    '''Random oversampling by non majority selection: the classes with less samples will be duplicated.
    This is an agnostic alternative to imbalanced-learn package which does not support nan/null/None values nor unencoded categorical variables.'''

    if not (y.value_counts() != y.value_counts().max()).any():
        # All classes are already balanced, nothing to do
        return X, y

    # Step 1: Vectorized sampling by non-majority selection (fast, will get approximately the correct amount of new samples per class to balance, but not exactly the same count, because it's probabilistic)
    yvalcount = y.value_counts()
    # Convert the counts into the remainder frequencies (what is needed to fill in each class to get equal with the majority). The majority class will get 0 by definition (ie, it will never be sampled).
    yfreq = (yvalcount.max() / yvalcount) - 1.0
    #yfreq = (yvalcount.max() - yvalcount) / yvalcount.max()
    # Normalize frequencies to 1 (to have probabilities)
    yfreq = yfreq / yfreq.sum()
    # Calculate the number of samples missing for all classes to be exactly equal
    yntogen = (yvalcount.max() * len(yvalcount)) - yvalcount.sum()
    # Generate a weight vector, where each index is a sample and the weight is the one corresponding to the yfreq for this class
    yfreq2 = y.replace(yfreq)
    # Sample!
    yidx = y.sample(n=yntogen, replace=True, weights=yfreq2, random_state=0)
    # Get the new samples in X and y
    Xnew = X.loc[yidx.index, :]
    ynew = y.loc[yidx.index]
    # Append the new samples to the original dataframes/series
    X2 = pd.concat([X, Xnew], ignore_index=True)
    y2 = pd.concat([y, ynew], ignore_index=True)

    # Step 2: Non-vectorized sampling by non-majority selection (slower because non vectorized, but allows to match exactly the same count for all classes)
    y2count = y2.value_counts()
    i = 0
    while (y2count != y2count.max()).any():  # loop until all classes have the same number of samples
        i += 1
        # Get any sample that is not of the majority class
        yrow = y2[~y2.isin(y2.mode(dropna=False))].sample(n=1, replace=True, random_state=i)
        # Duplicate it
        X2 = X2.append(X2.loc[yrow.index, :], ignore_index=True)
        y2 = y2.append(y2.loc[yrow.index], ignore_index=True)
        # Update the count
        y2count = y2.value_counts()

    return X2, y2

if oversample:
    print('Random oversampling for class balancing done!')
    print('Before oversampling:')
    print(y_train.value_counts())
    X_train, y_train = random_oversample(X_train, y_train)
    print('\n')
    print('After oversampling')
    print(y_train.value_counts())
    # Ensure the oversampling worked consistently for both X and y (should have the same number of samples in both)
    assert(len(X_train) == len(y_train))

In [ ]:
# Check that the last element was correctly duplicated 
lastx = '|'.join(str(x) for x in X_train.iloc[-1, :])
simx = X_train.apply(lambda x: '|'.join(str(xx) for xx in x), axis=1)
print(y_train.loc[simx == lastx])
X_train[simx == lastx]

In [ ]:
# DropOut-like oversampling, to duplicate samples but with values randomly nulled, so that the machine learning algo will hav to learn to predict with less features, and thus be more robust
dropout = True
n = 0.1 # percentage of the dataset to generate with randomly nullified features

#if dropout:
def dropout_oversampling(X, y, n=0.1, proba=0.1, nullarr=None, exclude=None):
    '''DropOut-like oversampling, to duplicate samples but with values randomly nulled, so that the machine learning algo will hav to learn to predict with less features, and thus be more robust
    Note: the sampling is uniform. Balance your classes before using a random oversampling if you want to balance and ensure all classes will have some nullified features, else the uniform distribution should more or less maintain the same class (im)balance without any impact. The goal of this procedure is not to balance out classes, but to ensure the machine learning can also work and have good performances with null features.
    X and y need to have the same indices
    n is the number of samples to add, it can be a float to be a ration compared to the length of X/y.
    proba is the probability of nulling PER feature/column, not for a whole row. If it > 1, then this will represent the amount of columns that should be nullified for sure (the choice will be random), eg: n=2 to nullify 2 columns of each sample.
    nullarr is a dictionary of all X's columns names as keys and the null values appropriate for each feature/column as values. If set to None, will try to autodetect based on column's dtype.
    exclude is the list of columns in X to exclude from nullifying.
    '''
    # For reproducibility
    #np.random.seed(0)

    # If n is a ratio, convert to an integer number of samples
    if n < 1.0:
        n = int(n * len(y))

    # Autodetect appropriate null values for each column's dtype
    if nullarr is None:
        nullarr = {}
        for col in X.columns:
            if X[col].dtype.kind == 'O':
                # Object or string
                nullarr[col] = None
            elif X[col].dtype.kind == 'f':
                # Float
                nullarr[col] = float('NaN')
            elif X[col].dtype.kind == 'i':
                # Integer
                if (X[col] == None).any():
                    # New Integer64 type supports None values, if we find any in the column, then we go for it
                    nullarr[col] = None
                else:
                    # Else we fall back to 0 (but might be wrong!)
                    nullarr[col] = 0
            else:
                raise(ValueError('Could not autodetect an appropriate null value for column %s dtype %s, please provide your own nullarr.' % (col, str(X[col].dtype))))

    # Calculate all probabilities of all features/columns and samples at once, this is faster (but if dataset is too big it can cause memory issues)
    #P = np.random.uniform(size=(n, X.shape[1]))
    for i in range(n):
        # Calculate the probability to null for all columns of next row
        P = np.random.uniform(size=(1, X.shape[1]))
        
        # Get the probability threshold
        if proba >= 1:
            # If it's an integer bigger than one, this is the number of columns to nullify, get the corresponding probability
            probathresh = np.sort(P)[0][int(proba-1)]
        else:
            # Else it's directly a probability threshold
            probathresh = proba

        # Select a sample randomly
        yidx = y.sample(n=1, random_state=i)
        # Get the sample
        Xnew = X.loc[yidx.index, :]
        ynew = y.loc[yidx.index]
        # Loop through each column
        for (col, _), p, nullval in zip(Xnew.items(), P[0], nullarr):
            # Skip excluded columns
            if exclude is not None and col in exclude:
                continue
            # Nullify if the probability is below the threshold
            elif p <= probathresh:
                Xnew[col] = nullarr[col]

        # Append the new dropout-modified sample
        X = X.append(Xnew, ignore_index=True)
        y = y.append(ynew, ignore_index=True)

    return X, y

if dropout:
    X_train, y_train = dropout_oversampling(X_train, y_train, n, 0.2)
    X_train = fillnaseparateclass(X_train) # encode None in object columns as strings, else it won't work for most machine learning packages
    assert len(X_train) == len(y_train)
    print('Dropout oversampling done!')
    print('New size:')
    print(len(X_train))
    print('Last 20 entries:')
    print(X_train.iloc[-20:, :])

In [ ]:
msno.matrix(X_train)

import imblearn
    # TODO: both encoded and non encoded datasets won't have the same resampling!

balanceclasses = True
if balanceclasses:
    # SMOTE-NC would be the best, because it interpolates numeric values and thus create new entries, but it does not yet support missing values, and using a placeholder imputation is incorrect: https://github.com/scikit-learn-contrib/imbalanced-learn/issues/157
    # Thus we instead use a RandomOverSampler, which will just duplicates as-is the samples to balance the power of each class, without interpolating new realistic samples
    oversampler = imblearn.over_sampling.RandomOverSampler(sampling_strategy='not majority', random_state=0) # if smote-nc, use kneighbors = 1 to avoid a majority vote (but then no interpolation!!! Should fix in code to avoid majority vote and just choose randomly, we don't want to do any data cleaning, we can do that afterward!)
    # Create a copy
    X_trainov = X_train.copy()
    X_train_encov = X_train_enc.copy()
    # Imputation of nan values to a fake value
    X_trainov[X_trainov.isnull()] = -999999999.999999
    X_train_encov[X_train_encov.isnull()] = -999999999.999999
    # Resample
    X_train, y_train = oversampler.fit_resample(X_train, y_train)
    X_train_enc, y_train_enc = oversampler.fit_resample(X_train_enc, y_train_enc)
    # Place back the null values exactly as they were (same type etc)
    X_trainov[X_train.isnull()] = X_train
    X_train_encov[X_train_enc.isnull()] = X_train_enc

In [ ]:
# Check class balance in training dataset
y_train.value_counts().apply(lambda x: x / len(y_train))

In [ ]:
# Check class balance in test dataset
y_test.value_counts().apply(lambda x: x / len(y_test))

### Data encoding (fitting and transform, after oversampling)

In [ ]:
# Learn encoding of categorical variables with LabelEncoder. It's not optimal but we hope the machine learning algo won't use any ordering artificially created by the label encoder.
from sklearn import preprocessing

# Encode the target y classes
enc_y = preprocessing.LabelEncoder()
enc_y.fit(pd.concat([y_orig, y_train]))

# Encode the categorical input features in X
enc_X = {}
enc_X_mapping = {}
X_full = pd.concat([X_orig, X_train])  # we add the training dataset because it might contain new values after oversampling (either because of interpolation if using SMOTE-NC, or because of new nullified values if the column was complete before)
X_full = fillnaseparateclass(X_full)  # make sure all nans are string, else the label encoder will choke
for col in categorical_columns:
    enc_x = preprocessing.LabelEncoder()
    print('Fitting to column: %s' % col)
    enc_x.fit(X_full[col])
    enc_X[col] = enc_x
    enc_X_mapping[col] = dict(zip((int(x) for x in enc_x.transform(enc_x.classes_)), enc_x.classes_))  # get the mapping, useful for later
    print(enc_X_mapping[col])
    print('\n')
del X_full

In [ ]:
#pd.concat([X_orig[col], X_train[col]])
X_orig[col]

In [ ]:
# Encode now
y_train_enc = enc_y.transform(y_train)
y_test_enc = enc_y.transform(y_test)

X_train_enc = fillnaseparateclass(X_train.copy())  # labelencoder unfortunately does not support None values, hence need to encode them as strings...
X_test_enc = fillnaseparateclass(X_test.copy())
for col in categorical_columns:
    X_train_enc[col] = enc_X[col].transform(X_train_enc[col])
    X_test_enc[col] = enc_X[col].transform(X_test_enc[col])

### XGBoost classifier

import xgboost as xgb

mod = xgb.XGBClassifier(
    gamma=1,                 
    learning_rate=0.01,
    max_depth=3,
    n_estimators=10000,                                                                    
    subsample=0.8,
    random_state=34
)

mod.fit(X_train, y_train)
predictions = mod.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, predictions))
print("score: {0:,.0f}".format(rmse))

### CatBoost classifier

In [ ]:
# https://www.kaggle.com/vchulski/dota-2-catboost-and-shap-explainer
#https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb
import catboost
model = catboost.CatBoostClassifier(custom_loss=['Accuracy'], random_seed=0, eval_metric='AUC', logging_level='Silent')


In [ ]:
X_train

In [ ]:
X_train['unified.diagnosis_worst']

In [ ]:
y_train

In [ ]:
# Fit CatBoost
pyver = 3
if pyver == 2:
    # For Python 2, CatBoost has currently a bug that makes it incapable of handling categorical features automatically, thus one needs to encode them beforehand
    # See https://github.com/catboost/catboost/issues/958
    model.fit(X_train_enc, y_train_enc,
        cat_features=categorical_columns,
        eval_set=(X_test_enc, y_test_enc),
        #logging_level='Verbose',  # you can uncomment this for text output
        plot=True #Uncomment and you'll see really great real time interactive graph
    )
else:
    # Python 3, CatBoost can handle categorical features automatically, no need for an encoder
    model.fit(fillnaseparateclass(X_train), y_train,
        cat_features=categorical_columns,
        eval_set=(fillnaseparateclass(X_test), y_test),
        #logging_level='Verbose',  # you can uncomment this for text output
        plot=True #Uncomment and you'll see really great real time interactive graph
    )

In [ ]:
from sklearn.metrics import accuracy_score

if pyver == 2:
    print('Accuracy on Training set: %f, on Test set: %f' % (accuracy_score(y_train_enc, model.predict(X_train_enc)), accuracy_score(y_test_enc, model.predict(X_test_enc))))
else:
    print('Accuracy on Training set: %f, on Test set: %f' % (accuracy_score(y_train, model.predict(X_train)), accuracy_score(y_test, model.predict(X_test))))

In [ ]:
from csg_fileutil_libs.model_evaluation_utils import model_evaluation_utils as meu

# Per class accuracy
# using library from https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/notebooks/Ch05_Building_Tuning_and_Deploying_Models/model_evaluation_utils.py as explained in https://towardsdatascience.com/explainable-artificial-intelligence-part-3-hands-on-machine-learning-model-interpretation-e8ebe5afc608
meu.display_model_performance_metrics(true_labels=y_train,
                                      predicted_labels=model.predict(X_train),
                                      classes=y_orig.unique())

In [ ]:
meu.display_model_performance_metrics(true_labels=y_test,
                                      predicted_labels=model.predict(X_test),
                                      classes=y_orig.unique())

In [ ]:
model.plot_tree(0)

In [ ]:
# Test with missing values where it was complete, to check if the classifier can still work
onetest = pd.DataFrame([['vs/uws', 1.0, 10, float('NaN'), 'M', True, '']], columns=X_train.columns)
model.predict(onetest)

In [ ]:
# TODO: save the model to be able to reload it later!

In [ ]:
# MICE imputer: the best approach but sklearn does NOT support categorical variables, so we would need to first use a labelencoder, but then the MICE imputer will infer an ordering in categorical variables where there is none

# explicitly require this experimental feature
#from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
#from sklearn.impute import IterativeImputer

#imp = IterativeImputer(sample_posterior=True, random_state=0)
#X_train_imp = imp.fit_transform(X_train)
#X_train_imp

In [ ]:
# Simple custom imputer: replace floats nan values with the median
# This is necessary for Skater to work
X_train_imp = X_train.copy()
X_test_imp = X_test.copy()
X_train_encimp = X_train_enc.copy()
X_test_encimp = X_test_enc.copy()
for col in X_orig.columns:
    if X_orig[col].dtype == 'float':
        for x_imp in (X_train_imp, X_test_imp, X_train_encimp, X_test_encimp):
            x_imp.loc[x_imp[col].isnull(), col] = x_imp[col].median()
print('Imputing done')

### Skater

In [ ]:
from skater import about
from packaging import version
skaterver = about.__version__
if version.parse(skaterver) < version.parse('1.1.2'):
    raise ImportError('Need Skater at least version 1.1.2 to use it here, current version installed is: %s. Use the conda build here: https://anaconda.org/derickl/skater' % skaterver)

In [ ]:
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

interpreter = Interpretation(X_train_imp, feature_names=list(X_orig.columns))

# If you get an OverflowError when using plot in the cell after, it's probably because you have nan values in your floats, see for a fix: https://github.com/oracle/Skater/issues/285
#interpreter.load_data(X_train, feature_names=list(X_orig.columns))
im_model = InMemoryModel(model.predict_proba,
                         examples=X_test_imp,
                         target_names=y_train.unique(),
                         #unique_values=y_train.unique(),
                         #model_type='classifier',
                        )

In [ ]:
# Plot barplot of features importance
plots = interpreter.feature_importance.plot_feature_importance(im_model, ascending=True)

In [ ]:
plots2 = interpreter.partial_dependence.plot_partial_dependence(list(X_orig.columns), im_model, with_variance=True)

In [ ]:
# From https://oracle.github.io/Skater/reference/interpretation.html#interpretable-rule-based
# Need to install skater >= 1.1.2, via this conda build: https://anaconda.org/derickl/skater
# Works only on label encoded categorical variables and without nans

from skater.util.dataops import convert_dataframe_to_dict, show_in_notebook

surrogate_explainer = interpreter.tree_surrogate(im_model, seed=0)
surrogate_explainer.fit(X_train_encimp, y_train, use_oracle=True, prune='post', scorer_type='default')
surrogate_explainer.plot_global_decisions(colors=['coral', 'lightsteelblue','darkkhaki'],
                                          file_name='simple_tree_pre.png')
show_in_notebook('simple_tree_pre.png', width=400, height=300)

In [ ]:
# Print all the predicted diagnoses, with and without encoding, if some are missing then there is a problem
print(np.unique(model.predict(X_train)))
print(np.unique(model.predict(X_train_encimp)))

In [ ]:
# TODO: learn my own decision tree with probabilities of decision and that supports categorical variables and empty values

In [ ]:
#from skater.core.global_interpretation.interpretable_models.brlc import BRLC

### Features importance: SHAP

In [ ]:
import skimage
if version.parse(skimage.__version__) < version.parse('0.14.2'):
    raise ImportError('scikit-image > v0.14.2 is required, but v%s is installed, please update (use `conda install -c conda-forge scikit-image`).' % skimage.__version__)

In [ ]:
import shap
shap.initjs()

In [ ]:
X_train[X_train.isnull().any(axis=1)]

In [ ]:
import catboost
# Workaround because shap does not support categorical features yet
shap_values = model.get_feature_importance(catboost.Pool(X_train, y_train, cat_features=categorical_columns), type=catboost.EFstrType.ShapValues)

expected_value = shap_values[0,-1]
shap_values2 = shap_values[:,:-1]

shap_values

In [ ]:
X_train2 = X_train.copy()
for col in X_train2.columns:
    if X_train2[col].dtype == 'str' or X_train2[col].dtype == 'object':
        X_train2.loc[X_train2[col].isnull(), col] = 'None'

In [ ]:
#explainer = shap.TreeExplainer(model)
#shap_values = explainer.shap_values(X_train2, approximate=True)

In [ ]:
# Install latest version of SHAP else it won't work (not for TreeExplainer at least, but KernelExplainer should work)
# See for workarounds https://github.com/slundberg/shap/issues/662
# and the fix https://github.com/slundberg/shap/issues/736
explainer = shap.KernelExplainer(model.predict_proba, X_train)
#shap_values = explainer.shap_values(X_train)
shap_values = model.get_feature_importance(catboost.Pool(X_train, y_train, cat_features=categorical_columns), type='ShapValues')
print(shap_values.shape)

# visualize the first prediction's explanation
# use link logit to visualize probabilities instead of shap values. This is yet unavailable for summary plot: https://github.com/slundberg/shap/issues/756
shap.force_plot(explainer.expected_value[0], shap_values[0], feature_names=X_orig.columns, out_names=list(y_train.unique()), link='logit')

In [ ]:
# All features pairwise shap values interactions
# THIS IS WRONG! it's not the pairwise shap values interactions!
interaction_values = model.get_feature_importance(catboost.Pool(X_train, y_train, cat_features=categorical_columns), type='Interaction')
print(interaction_values.shape)
print(interaction_values)
shap.summary_plot(shap_values[:,:-1,:-1], features=X_train, plot_type='dot')

In [ ]:
# Summarize the effects of all the features in a multi-class barplot
# See for more explanations on how it was done: https://github.com/slundberg/shap/issues/750
# To understand how to interpret this graphic, see: https://github.com/slundberg/shap/issues/367
# TODO: fix order of class names, not sure this is the order used by catboost!

# Transpose the 3D numpy ndarray (shape [#samples, #classes, #features]) so we place the classes first
#original_shape = shap_values.shape
#shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1], order='C') # not the correct way of doing it, you will get the same mean SHAP value for all classes!
shap_values_transposed = shap_values.transpose(1, 0, 2)
assert shap_values[0, 1, 2] == shap_values_transposed[1, 0, 2]  # just check we've done it right
print(shap_values_transposed.shape)

# Then we convert the transposed shap values into a list, where each element will be a 2D numpy matrix of shape [#samples, #features] as expected by the shap package
shap.summary_plot(list(shap_values_transposed[:,:,:-1]), features=X_train, class_names=y_train.unique(), plot_type='bar')

In [ ]:
# summarize the effects of all the features for each class output
for yclass in range(len(shap_values_transposed)):
    plt.figure()
    shap.summary_plot(shap_values_transposed[yclass,:,:-1], features=X_train_enc, plot_type='violin')

In [ ]:
# summarize the effects of all the features for each class output
for yclass in range(len(shap_values_transposed)):
    plt.figure()
    shap.summary_plot(shap_values_transposed[yclass,:,:-1], features=X_train_enc, plot_type='dot') # color_bar_label change to class name

In [ ]:
shap.dependence_plot('unified.diagnosis_worst', shap_values_transposed[0, :, :-1], X_train_enc, display_features=X_train, # display_features allows to specify string labels for the features, it should be the non-encoded dataset, so the same matrix size as for the argument "features"
                    x_jitter=0.2, interaction_index='unified.etiology') #The index of the feature used to color the plot. The name of a feature can also be passed as a string. If “auto” then shap.common.approximate_interactions is used to pick what seems to be the strongest interaction (note that to find to true stongest interaction you need to compute the SHAP interaction values).

### ELI5

In [ ]:
# TODO: https://eli5.readthedocs.io/en/latest/_modules/eli5/catboost.html

In [ ]:
# TODO: use permutation of ELI5

### BESTree BESTForest classifier